#### 제주도 핫플레이스 인스타그램에서 크롤링하기

##### matplotlib 한글설정

##### 셀레니움 설정

In [4]:
import matplotlib
# from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc, rcParams
import seaborn as sns

font_path = 'C:/Windows/Fonts/NanumGothic.ttf'
font = font_manager.FontProperties(fname=font_path).get_name() # 'NanumGothic'
rc('font', family=font)
rcParams['axes.unicode_minus'] = False 

In [6]:
# 셀레니움
# 모듈 임포트
from selenium import webdriver

driver = webdriver.Chrome()
url = 'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91/'
driver.get(url)

##### 첫번째 게시글 찾아서 클릭하기


In [7]:
from selenium.webdriver.common.by import By
import time

def select_first_post(driver):
    first = driver.find_elements(By.CLASS_NAME, '_aagw')[0]
    first.click()
    time.sleep(3)

select_first_post(driver)

##### 게시글 정보 가져오기

In [8]:
import re # RegularExpression 정규식 처리 모듈
from bs4 import BeautifulSoup
import unicodedata  # 유니코드 데이터 처리 모듈

def get_content(driver):
    # 1. 데이터가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 2. 본문내용 찾아오기
    try:
        content = soup.select('div._a9zr')[0].text   # _a9zr은 추후 변경됨(인스타그램 정책)
    except:
        content = ''

    # 3. 본문내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)

    # 4. 작성일자 정보
    try:
        date = soup.select('time._aaqe')[0]['datetime'][:10] # _aaqe은 추후 변경됨(인스타그램 정책)
    except:
        date = ''

    # 5. 좋아요수 가져오기
    try:
        # x193iq5w등은 추후 변경됨(인스타그램 정책)
        like = int(soup.select('span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp > span')[0].text)
    except:
        like = 0

    # 6. 위치정보
    # 정규식 학습필수
    # _aagv 마지막 인덱스를 찾으면 된다
    # imgs = soup.select('div._aatk._aatl._aatm div._aagv > img')[0]['alt']
    address = ''
    try:
        regex = r'(\w+[시,읍]\s*)?' \
                r'?(\w+\d*\w*[로,리,길]\s*)' \
                r'?(\w*\d+-?\d*)?'

        place = re.findall(regex, content)
        for pl in place:
            if pl[0] != '' and pl[1] != '' and pl[2] != '':
                address = f'{pl[0]}{pl[1]}{pl[2]}'
    except:
        address = ''

    # 7. 정보 저장
    data = [content, date, like, address, tags]
    return data

get_content(driver)

['bbo_muksta한입 먹는 순간 여기 티라미수밖에 못먹는다. 큰일이네 이거… #애월아시시제주도에서 오션뷰에 분위기도 좋고 맛있는 디저트까지 다 갖춘 곳이있다. 애월 아시시. 티라미수 찐맛집이라 현지인도 자주가는 곳. 꼭 가서 먹고 포장까지 야무지게 하자. 티라미수 근본이다. 촉촉한 시트 꾸덕크리미한 식감,찌인한 크림치즈 풍미,은은한 커피향까지 완벽 그 자체. 흑임자 티라미수도 쫀득하게 씹히는 떡이 식감을 다채롭게 해주고 맛도 찌인해서 강력추천. 보리개역 크림라떼는 고소함이 일품이다. 꼬소한 미숫가루향과 라떼 그리고 바삭한 크런치까지 너무 매력적이라 계속 끌리는 맛이니 꼭 먹자. ✔️위치 : 제주 제주시 애월읍 애월해안로 474 지하1층✔️메뉴 : 티라미수(8500),보리개역 크림라떼(7500)✔️ @assisi_aewol #티라미수맛집 #디저트맛집 #애월맛집 #제주도카페15시간',
 '2023-07-09',
 702,
 '애월읍 애월해안로 474',
 ['#애월아시시제주도에서', '#티라미수맛집', '#디저트맛집', '#애월맛집', '#제주도카페15시간']]

In [9]:
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, ('._aaqg._aaqh'))
    right.find_element(By.XPATH, '//button')
    right.click()
    time.sleep(3)

move_next(driver)

##### 자동 크롤링 구현

In [27]:
# 모듈 임포트 생략

driver = webdriver.Chrome()
driver.get('https://www.instagram.com')
time.sleep(2.0)

In [28]:
# 아이디, 패스워드 입력해서 자동 로그인하기
id = '01036825261'
input_id = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[0]
input_id.clear()
input_id.send_keys(id)
password = 'cdt92f278*'
input_pw = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(2.0)

In [20]:
# 알림설정 버튼 클릭
alarm_btn = driver.find_element(By.CSS_SELECTOR, '._a9--._a9_0')
alarm_btn.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"._a9--._a9_0"}
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00BCA813+48355]
	(No symbol) [0x00B5C4B1]
	(No symbol) [0x00A65358]
	(No symbol) [0x00A909A5]
	(No symbol) [0x00A90B3B]
	(No symbol) [0x00ABE232]
	(No symbol) [0x00AAA784]
	(No symbol) [0x00ABC922]
	(No symbol) [0x00AAA536]
	(No symbol) [0x00A882DC]
	(No symbol) [0x00A893DD]
	GetHandleVerifier [0x00E2AABD+2539405]
	GetHandleVerifier [0x00E6A78F+2800735]
	GetHandleVerifier [0x00E6456C+2775612]
	GetHandleVerifier [0x00C551E0+616112]
	(No symbol) [0x00B65F8C]
	(No symbol) [0x00B62328]
	(No symbol) [0x00B6240B]
	(No symbol) [0x00B54FF7]
	BaseThreadInitThunk [0x75AF6BA9+25]
	RtlGetFullPathName_UEx [0x77128F9F+1215]
	RtlGetFullPathName_UEx

In [ ]:
# 제주도맛집 찾기 함수
def insta_searching(word):
    url = f'https://www.instagram.com/explore/tags/{word}'
    return url

In [ ]:
# 크롤링 시작하기
word = '제주도맛집'
url = insta_searching(word)
driver.get(url)
time.sleep(10.0)

select_first_post(driver)

results = []

target = 500
count = 0
for i in range(target):
    count += 1
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2.0)
        move_next(driver)
#     print(count)

# print(len(results))

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('./1_crawling_jejudoHotplaces.xlsx', index=False)